# 7506 - Trabajo Práctico 2

---

## Introducción

### Librerías

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.metrics import *
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import matplotlib.pyplot as plt
from matplotlib import style
from joblib import load
import sklearn as sk

2022-11-29 02:03:05.068389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 02:03:05.737701: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fede/anaconda3/lib/
2022-11-29 02:03:05.737804: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fede/anaconda3/lib/
2022-11-29 02:03:05.737811: W tensorflow/compiler/tf2tensorrt/utils/py_utils

In [2]:
# Dataset Train preprocesado
ds_train = pd.read_csv('datasets/tp1-train_id.csv')
ds_train = ds_train.drop(['Unnamed: 0'], axis=1)
ds_train.head()

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,1PYAQFsAktkYlJpcdrH9uQ==,2021-03-24,2021-05-26,-34.589481,-58.389144,Recoleta,Departamento,4.0,3.0,192.0,170.0,580000.0,Balcón terraza y totalmente reciclado en Recol...
1,Qgy7pmJ4NulNrdbxa8a7MQ==,2021-12-13,2021-12-23,-34.555436,-58.475303,Nuñez,Departamento,3.0,2.0,83.0,83.0,100000.0,Departamento - Nuñez
2,QO1fBv/XWsHIqP59T0Xngg==,2021-09-24,2022-03-09,-34.577124,-58.475892,Villa Urquiza,Casa,4.0,3.0,150.0,105.0,229000.0,Casa en venta 4Amb c/patio y terraza - Urquiza R
3,9yo77b8vzfYAXyFJHVhLiw==,2021-07-16,9999-12-31,-34.639306,-58.368501,Barracas,Departamento,2.0,1.0,122.0,75.0,195000.0,Departamento - Barracas
4,HEyW45D+cF+tLUGaQPDgXg==,2021-08-30,2022-01-27,-34.563572,-58.439407,Palermo,Departamento,3.0,2.0,50.0,50.0,115000.0,"Dpto. 3 ambientes, con cocina independiente y ..."


In [3]:
# Dataset Test preprocesado
ds_test = pd.read_csv('datasets/tp1-test_id.csv')
ds_test = ds_test.drop(['Unnamed: 0'], axis=1)
ds_test

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,poGOTUlIzEZKiMYhXKdYQQ==,2021-01-15,2021-04-29,-34.577306,-58.434056,Palermo,Departamento,3.0,2.0,68.0,68.0,158000.0,Departamento 3 Ambientes Reciclado Escritorio ...
1,/vmZVmXMvf5tTlS14gcC8w==,2021-09-01,2022-09-10,-34.558066,-58.473512,Belgrano,Casa,6.0,5.0,341.0,206.0,399000.0,Casa belgrano jardin pileta luminoso doble co...
2,ko5Ol+Mv8KN8Kglz1k/ubA==,2021-12-08,2021-12-19,-34.545531,-58.474900,Saavedra,PH,1.0,1.0,20.0,20.0,45000.0,Dpto PH Saavedra 1amb sin expe P.Baja a la cal...
3,/Z37wOCvzjGlLKMdOFky+Q==,2021-04-26,2021-06-05,-34.605870,-58.457702,Caballito,Departamento,3.0,2.0,60.0,52.0,149000.0,"Venta, Departamento, 3 ambientes, Balcón, coch..."
4,RXg51b1AdPe+UXZkpRNEqg==,2021-01-07,2021-02-10,-34.587415,-58.438877,Palermo,Departamento,2.0,1.0,99.0,58.0,210000.0,U$S 210.000 - 2 amb - Palermo - Niceto Vega 52...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12061,zMNYVtyC8ZPLlHJDVEzcJg==,2021-05-22,2021-06-05,-34.562450,-58.454040,Belgrano,Departamento,3.0,2.0,76.0,70.0,204000.0,Venta Departamento 3 amb con Balcón Dependenci...
12062,TI8pKfusTlHy5oPeb/ZBWQ==,2021-03-02,2021-03-10,-34.598424,-58.440173,Villa Crespo,Departamento,4.0,3.0,136.0,123.0,299000.0,"4 AMB C/DEP, COCHERA, PISO ALTO! VILLA CRESPO"
12063,y8Uvsi21S5hPqQT7OeAwAw==,2021-08-10,9999-12-31,-34.639597,-58.369096,Barracas,Departamento,2.0,1.0,105.0,92.0,230000.0,Departamento - Barracas
12064,R1uGoXBwjP52Z1ZAwqQ3dQ==,2021-01-11,2021-02-08,-34.574611,-58.492324,Villa Urquiza,Departamento,2.0,1.0,67.0,59.0,119538.0,Departamento - Villa Urquiza


## Procesamiento del lenguaje natural

### Ampliación del dataset

Este dataset incluye descripciones de las propiedades del otro dataset. Veremos como podemos extraer información de estas descripciones.

In [4]:
descriptions_dataset = pd.read_csv('datasets/properati_argentina_2021_decrip.csv')
descriptions_dataset.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...


Tomaremos las descripciones correspondientes a los datasets de train y test

In [5]:
descriptions_train = descriptions_dataset[descriptions_dataset.id.isin(ds_train.id)].copy()
descriptions_test = descriptions_dataset[descriptions_dataset.id.isin(ds_test.id)].copy()
descriptions_train.shape, descriptions_test.shape

((60705, 2), (12066, 2))

#### Análisis de sentimientos - Tecnica Minqing Hu y Bing Liu

Una forma de analizar el sentimiento de un de un texto es considerando su sentimiento como la suma de los sentimientos de cada una de las palabras que lo forman.

Para el analisis de sentimiento nos guiamos del analisis realizado en esta pagina: https://www.cienciadedatos.net/documentos/py25-text-mining-python.html

Utilizamos algunas funciones de tokenizacion y limpieza de ahi con alguna sutil modificacion para nuestro caso de uso en particular.

In [6]:
def limpiar_tokenizar(texto):
    '''
    Esta función limpia y tokeniza el texto en palabras individuales.
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''

    # Se convierte todo el texto a minúsculas
    nuevo_texto = str(texto).lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\¡\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex, ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep=' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]

    return (nuevo_texto)

In [7]:
# se aplica la función de limpieza a train y test y tokenización a cada descripcion

tokenized_train = pd.concat([descriptions_train.id, descriptions_train['property_description'].apply(limpiar_tokenizar)], axis=1)
tokenized_test = pd.concat([descriptions_test.id, descriptions_test['property_description'].apply(limpiar_tokenizar)], axis=1)
tokenized_train.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,"[corredor, responsable, maria, alejandra, geno..."
3,gn55JPmcjftBV/at1a7fPg==,"[unico, ph, de, dormitorios, en, impecable, es..."
11,uH5H+RfQM87Qtrb13+T4hA==,"[venta, ph, ambientes, paternal, br, br, cálid..."
16,n7NJmavbeNUlyKufoGRr+g==,"[oportunidad, br, acceso, por, escalera, ampli..."
153,lZyClw0vqQvUiJkIemcg8A==,"[departamento, piso, ambientes, con, dependenc..."


Separamos los tokens según ids tanto en train como en test.

In [8]:
tokens_train = tokenized_train.explode(column='property_description')
tokens_train = tokens_train.rename(columns={'property_description': 'token'})
tokens_train.reset_index(inplace=True, drop=True)
tokens_train.head()

,id,token
0,BZCeiGkJr1WBUi6pKJQwJQ==,corredor
1,BZCeiGkJr1WBUi6pKJQwJQ==,responsable
2,BZCeiGkJr1WBUi6pKJQwJQ==,maria
3,BZCeiGkJr1WBUi6pKJQwJQ==,alejandra
4,BZCeiGkJr1WBUi6pKJQwJQ==,genoveva


In [9]:
# Replicamos en test.
tokens_test = tokenized_test.explode(column='property_description')
tokens_test = tokens_test.rename(columns={'property_description': 'token'})
tokens_test.reset_index(inplace=True, drop=True)

In [10]:
tokens_train.shape, tokens_test.shape

((11156503, 2), (2242184, 2))

Vemos que tenemos 11 millones de palabras en train y 3 millones en test

Notamos que las preposiciones no son relevantes para entender que atributo podria ser mejor para expandir el datast, asi que decidimos agregarlas como stopwords.

Tampoco van a variar mucho el analisis de sentimiento realizado en este trabajo.

In [11]:
## listado de stopwords

preposiciones = ["a", "ante", "bajo", "cabe", "con", "contra", "de", "desde", "durante", "en", "entre", "hacia",
                 "hasta", "mediante", "para", "por", "según", "sin", "sobre", "tras", "vía"]

stop_words = []

stop_words += preposiciones

# filtrado para excluir stopwords
tokens_train = tokens_train[~(tokens_train["token"].isin(stop_words))]

tokens_test = tokens_test[~(tokens_test["token"].isin(stop_words))]

Agregamos un lexicon en español de esta pagina: https://github.com/jboscomendoza/lexicos-nrc-afinn

In [12]:
# lexicon sentimientos
lexicon = pd.read_csv('datasets/lexico_nrc.csv')
lexicon

,palabra,sentimiento,word
0,ciegamente,negativo,blindly
1,ciegamente,tristeza,blindly
2,a distancia,negativo,aloof
3,a hurtadillas,sorpresa,stealthily
4,a la deriva,anticipación,adrift
...,...,...,...
15302,zumbido,anticipación,buzz
15303,zumbido,miedo,buzz
15304,zumbido,positivo,buzz
15305,zumbido,negativo,drone


In [13]:
def mappear_valores_sentimiento(s):
    # 1 Positivo
    # 0 Neutro
    # -1 Negativo
    sentimiento_numerico = 0
    if str(s) in ['negativo', 'tristeza', 'miedo', 'enfado', 'tristeza', 'asco']:
        sentimiento_numerico = -1
    if str(s) in ['sorpresa', 'positivo', 'confianza', 'alegría']:
        sentimiento_numerico = 1
    if str(s) in ['anticipación']:
        sentimiento_numerico = 0

    return sentimiento_numerico


In [14]:
lexicon['sentimiento'] = lexicon['sentimiento'].apply(lambda x: mappear_valores_sentimiento(x))
lexicon[['sentimiento']].head()

,sentimiento
0,-1
1,-1
2,-1
3,1
4,0


In [15]:
tokens_train[tokens_train.token.isin(lexicon.palabra)].shape, tokens_test[tokens_test.token.isin(lexicon.palabra)].shape

((1305910, 2), (261835, 2))

In [16]:
lexicon

,palabra,sentimiento,word
0,ciegamente,-1,blindly
1,ciegamente,-1,blindly
2,a distancia,-1,aloof
3,a hurtadillas,1,stealthily
4,a la deriva,0,adrift
...,...,...,...
15302,zumbido,0,buzz
15303,zumbido,-1,buzz
15304,zumbido,1,buzz
15305,zumbido,-1,drone


Tenemos un millon y medio de coincidencias con el lexicón en train. Usaremos estos sentimientos para puntuar las propiedades

In [17]:
# sentimiento promedio de cada descripcion
tokens_sentimientos_train = pd.merge(
    left=tokens_train,
    right=lexicon,
    left_on="token",
    right_on="palabra",
    how="inner"
)
tokens_sentimientos_train = tokens_sentimientos_train.drop(columns=["palabra", "word"])

tokens_sentimientos_test = pd.merge(
    left=tokens_test,
    right=lexicon,
    left_on="token",
    right_on="palabra",
    how="inner"
)
tokens_sentimientos_test = tokens_sentimientos_test.drop(columns=["palabra", "word"])

tokens_sentimientos_train.head()

,id,token,sentimiento
0,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
1,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
2,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
3,boCXaj+OYbucoJeinbtaOg==,responsable,1
4,boCXaj+OYbucoJeinbtaOg==,responsable,1


Ahora calcularemos el puntaje para cada propiedad como la suma de los sentimientos.

In [18]:
score_train = tokens_sentimientos_train[["id", "token", "sentimiento"]].groupby(["id"]).sum().reset_index()

score_test = tokens_sentimientos_test[["id", "token", "sentimiento"]].groupby(["id"]).sum().reset_index()

In [19]:
score_train

,id,sentimiento
0,+++LrfX9dl/rsnbNTgVaSw==,24
1,++170jViMoXTsnGwVScNsA==,5
2,++20Jf3Ha2Epz88cJxEmJw==,10
3,++2AcJNn9KYBeuOard+DZw==,18
4,++5o0X5nwMy3BtfAvsFAaA==,-2
...,...,...
60215,zziqoG25k9M8rIgzpKwFwg==,29
60216,zzjbYucw7Rj0OtPfDWrsuA==,15
60217,zzkHuJd+M8QH9JSoKm9e0A==,19
60218,zzrpS8TqDA6px1h/DuQdmw==,20


In [20]:
score_train.sentimiento.max()

216

La mejor propiedad tiene un puntaje de 216. Analizaremos un poco las descripciones de las mejores y peores.

In [21]:
top5_positivas = score_train.sort_values(by='sentimiento', ascending=False).head(5)
top5_positivas

,id,sentimiento
55723,vJRa8QKk0d9i9L4Rk6Bvng==,216
42789,hWcrJ6XkTCTGVo80jBvI1g==,185
25130,Ordgp/sLvmhD+RwAtErcvw==,185
9679,8HZPdHHB8QMCi2U/iJkYmw==,185
31493,Vi4Kbdne9us6XT3kpmSmgw==,185


In [22]:
descriptions_train.property_description.iloc[top5_positivas.index]

431740    Edificio atrás del Alvear Palace Hotel, muy co...
329471    Interesante PH para Reciclar<br>Muy buenas dim...
191258    Muy buen departamento en Gavilán 975, muy lumi...
72690     Corredor Responsable: Viviana Uzcudun - CSI 58...
243161    Corredor Responsable: Cristian Arnal Ponti - C...
Name: property_description, dtype: object

In [23]:
score_train.sentimiento.min()

-37

In [24]:
top5_negativas = score_train.sort_values(by='sentimiento').head(5)
top5_negativas

,id,sentimiento
21188,KWFIk6SR8JFrfLNsfJcoPg==,-37
38471,d2ghEGP8mv2R8W1wvl47yA==,-25
58752,yTzDOa/iBUD6x1xW0p+8jQ==,-24
58838,yZ1q8SipMwHAKNK0xa1ApQ==,-22
54017,tT6oySFAhaUtHOUoCPT5nQ==,-20


In [25]:
descriptions_train.iloc[top5_negativas.index].property_description

161623    Corredor Responsable: Maria García Herz -  CPI...
295727    Gran patio terraza de uso exclusivo y parrilla...
448163    Corredor Responsable: JAVIER BURGUE DIAZ CPI 7...
448992    Corredor Responsable: Pamela Suarez CUCICBA 53...
419477    La propiedad está ubicada estratégicamente en ...
Name: property_description, dtype: object

descriptions_test.head()Como es claro, la gente que publica la venta de una propiedad va a tratar de expresar la mejor publicacion y descripcion posible. Es por eso que tenemos una tasa altisima de positividad. No buscamos hacer un analisis tan profundo de las descripciones sino crear un puntaje relativamente estandarizado para poder usar la descripción como feature.

In [26]:
def perfil_sentimientos(title, df):
    print(title)
    print(f"Positivos: {round(100 * np.mean(df.sentimiento > 0), 2)}")
    print(f"Neutros  : {round(100 * np.mean(df.sentimiento == 0), 2)}")
    print(f"Negativos: {round(100 * np.mean(df.sentimiento < 0), 2)}")


perfil_sentimientos("Train: ", score_train)
print()
perfil_sentimientos("Test: ", score_test)

Train: 
Positivos: 93.41
Neutros  : 1.68
Negativos: 4.92

Test: 
Positivos: 93.57
Neutros  : 1.6
Negativos: 4.84


Finalmente, agregaremos nuestro puntaje como columnas nuevas del dataset.

In [27]:
ds_test

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,poGOTUlIzEZKiMYhXKdYQQ==,2021-01-15,2021-04-29,-34.577306,-58.434056,Palermo,Departamento,3.0,2.0,68.0,68.0,158000.0,Departamento 3 Ambientes Reciclado Escritorio ...
1,/vmZVmXMvf5tTlS14gcC8w==,2021-09-01,2022-09-10,-34.558066,-58.473512,Belgrano,Casa,6.0,5.0,341.0,206.0,399000.0,Casa belgrano jardin pileta luminoso doble co...
2,ko5Ol+Mv8KN8Kglz1k/ubA==,2021-12-08,2021-12-19,-34.545531,-58.474900,Saavedra,PH,1.0,1.0,20.0,20.0,45000.0,Dpto PH Saavedra 1amb sin expe P.Baja a la cal...
3,/Z37wOCvzjGlLKMdOFky+Q==,2021-04-26,2021-06-05,-34.605870,-58.457702,Caballito,Departamento,3.0,2.0,60.0,52.0,149000.0,"Venta, Departamento, 3 ambientes, Balcón, coch..."
4,RXg51b1AdPe+UXZkpRNEqg==,2021-01-07,2021-02-10,-34.587415,-58.438877,Palermo,Departamento,2.0,1.0,99.0,58.0,210000.0,U$S 210.000 - 2 amb - Palermo - Niceto Vega 52...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12061,zMNYVtyC8ZPLlHJDVEzcJg==,2021-05-22,2021-06-05,-34.562450,-58.454040,Belgrano,Departamento,3.0,2.0,76.0,70.0,204000.0,Venta Departamento 3 amb con Balcón Dependenci...
12062,TI8pKfusTlHy5oPeb/ZBWQ==,2021-03-02,2021-03-10,-34.598424,-58.440173,Villa Crespo,Departamento,4.0,3.0,136.0,123.0,299000.0,"4 AMB C/DEP, COCHERA, PISO ALTO! VILLA CRESPO"
12063,y8Uvsi21S5hPqQT7OeAwAw==,2021-08-10,9999-12-31,-34.639597,-58.369096,Barracas,Departamento,2.0,1.0,105.0,92.0,230000.0,Departamento - Barracas
12064,R1uGoXBwjP52Z1ZAwqQ3dQ==,2021-01-11,2021-02-08,-34.574611,-58.492324,Villa Urquiza,Departamento,2.0,1.0,67.0,59.0,119538.0,Departamento - Villa Urquiza


In [28]:
ds_train = pd.merge(ds_train, score_train, on='id')
ds_train.rename(columns={'sentimiento': 'score_sentimientos'}, inplace=True)
ds_test = pd.merge(ds_test, score_test, on='id')
ds_test.rename(columns={'sentimiento': 'score_sentimientos'}, inplace=True)

In [29]:
ds_train.head()

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,score_sentimientos
0,1PYAQFsAktkYlJpcdrH9uQ==,2021-03-24,2021-05-26,-34.589481,-58.389144,Recoleta,Departamento,4.0,3.0,192.0,170.0,580000.0,Balcón terraza y totalmente reciclado en Recol...,13
1,Qgy7pmJ4NulNrdbxa8a7MQ==,2021-12-13,2021-12-23,-34.555436,-58.475303,Nuñez,Departamento,3.0,2.0,83.0,83.0,100000.0,Departamento - Nuñez,11
2,QO1fBv/XWsHIqP59T0Xngg==,2021-09-24,2022-03-09,-34.577124,-58.475892,Villa Urquiza,Casa,4.0,3.0,150.0,105.0,229000.0,Casa en venta 4Amb c/patio y terraza - Urquiza R,24
3,9yo77b8vzfYAXyFJHVhLiw==,2021-07-16,9999-12-31,-34.639306,-58.368501,Barracas,Departamento,2.0,1.0,122.0,75.0,195000.0,Departamento - Barracas,34
4,HEyW45D+cF+tLUGaQPDgXg==,2021-08-30,2022-01-27,-34.563572,-58.439407,Palermo,Departamento,3.0,2.0,50.0,50.0,115000.0,"Dpto. 3 ambientes, con cocina independiente y ...",44


#### Tecnica Regex

Revisamos las siguientes paginas para entender cuales son los ammenities mas buscados en CABA, y en Argentina en general.

https://www.iprofesional.com/negocios/371702-cuales-son-los-amenities-mas-exoticos-de-edificios-en-argentina

https://www.baenegocios.com/sociedad/Ranking-de-amenities-los-servicios-que-mas-pesan-al-comprar-una-propiedad-20220119-0068.html

https://www.forbesargentina.com/negocios/amenities-servicios-mas-demandados-argentinos-comprar-una-propiedad-n11901

Dichos ammenities parecen hacer que la propiedad cotice entre un 15% y un 20% más que el precio de venta.

Sacando un promedio y haciendo un top-5 ranking, podemos notar que los mas relevantes son:

- Garage/Estacionamiento
- Pileta
- Jardin/Espacio al aire libre
- Parrilla
- SUM (Gimnasio/Spa/Sauna)

Al buscar estos datos, podriamos tratar de entender si el precio resulta mayor, contra una propiedad de similares caracteristicas pero sin estos ammenities y a partir de eso, entender que % varía del precio de venta original.

In [30]:
def calculate_freq(feature, regex):
    freq = descriptions_train.property_description.str.contains(regex, regex=True).sum()
    print(
        f"Los anuncios de propiedades que tienen la feature {feature} son: {freq} y representan el {freq * 100 // len(descriptions_train)}% de los datos")

##### Amenities

In [31]:
garage = re.compile(r"\s*garage|garaje|estacionamiento|parking")
calculate_freq("Garage", garage)

Los anuncios de propiedades que tienen la feature Garage son: 1079 y representan el 1% de los datos


In [32]:
pileta = re.compile(r"\s*pileta")
calculate_freq("Pileta", pileta)

Los anuncios de propiedades que tienen la feature Pileta son: 3985 y representan el 6% de los datos


In [33]:
jardin = re.compile(r"\s*jardin|espacio verde")
calculate_freq("Jardín", jardin)

Los anuncios de propiedades que tienen la feature Jardín son: 1173 y representan el 1% de los datos


In [34]:
parrilla = re.compile(r"\s*parrilla|bbq")
calculate_freq("Parrilla", parrilla)

Los anuncios de propiedades que tienen la feature Parrilla son: 10787 y representan el 17% de los datos


In [35]:
sum = re.compile(r"\s*zoom|sum|gimansio|spa")
calculate_freq("SUM", sum)

Los anuncios de propiedades que tienen la feature SUM son: 22641 y representan el 37% de los datos


In [36]:
balcon = re.compile(r"\s*balcon|balcón")
calculate_freq("Balcón", balcon)

Los anuncios de propiedades que tienen la feature Balcón son: 28807 y representan el 47% de los datos


Otro aspecto interesante que dejamos fuera del análisis es a que tipo de vivienda pertenece cada ammenity. Y si donde encontramos una amenity en particular, encontramos consecuentemente otra. Por ejemplo, una casa con jardin y parrila y/o pileta. De esta manera podriamos tratar de determinar el costo de cada ammenity o como afecta al precio.

Por último, construiremos columnas booleanas para los mejores features y las agregaremos a nuestros datasets. Usaremos parrilla, sum, balcón y pileta.

In [37]:
amenities_train = pd.DataFrame({
    'id': descriptions_train.id,
    'pileta': descriptions_train.property_description.str.contains(pileta, regex=True),
    'parrilla': descriptions_train.property_description.str.contains(parrilla, regex=True),
    'balcon': descriptions_train.property_description.str.contains(balcon, regex=True),
    'sum': descriptions_train.property_description.str.contains(sum, regex=True)
})
amenities_train.head()

,id,pileta,parrilla,balcon,sum
0,BZCeiGkJr1WBUi6pKJQwJQ==,False,False,False,False
3,gn55JPmcjftBV/at1a7fPg==,False,False,False,False
11,uH5H+RfQM87Qtrb13+T4hA==,False,False,False,False
16,n7NJmavbeNUlyKufoGRr+g==,False,False,False,False
153,lZyClw0vqQvUiJkIemcg8A==,False,False,True,True


In [38]:
# Replicamos lo mismo en test
amenities_test = pd.DataFrame({
    'id': descriptions_test.id,
    'pileta': descriptions_test.property_description.str.contains(pileta, regex=True),
    'parrilla': descriptions_test.property_description.str.contains(parrilla, regex=True),
    'balcon': descriptions_test.property_description.str.contains(balcon, regex=True),
    'sum': descriptions_test.property_description.str.contains(sum, regex=True)
})
amenities_test.head()

,id,pileta,parrilla,balcon,sum
2,TUL813tXchVBlHUsfaA6DA==,False,True,True,False
151,ns8QWxBXYIdnEccdiu3qog==,False,False,False,False
154,AZJX5ab3sBKZK1bSC3zW9A==,False,False,False,False
336,Ta6ZlfqWQbNbo3jVCHHDVQ==,False,False,True,True
341,TFL4KKRhKUuOS9QiO4VM+g==,False,False,False,False


In [39]:
ds_test

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,score_sentimientos
0,poGOTUlIzEZKiMYhXKdYQQ==,2021-01-15,2021-04-29,-34.577306,-58.434056,Palermo,Departamento,3.0,2.0,68.0,68.0,158000.0,Departamento 3 Ambientes Reciclado Escritorio ...,29
1,/vmZVmXMvf5tTlS14gcC8w==,2021-09-01,2022-09-10,-34.558066,-58.473512,Belgrano,Casa,6.0,5.0,341.0,206.0,399000.0,Casa belgrano jardin pileta luminoso doble co...,16
2,ko5Ol+Mv8KN8Kglz1k/ubA==,2021-12-08,2021-12-19,-34.545531,-58.474900,Saavedra,PH,1.0,1.0,20.0,20.0,45000.0,Dpto PH Saavedra 1amb sin expe P.Baja a la cal...,38
3,/Z37wOCvzjGlLKMdOFky+Q==,2021-04-26,2021-06-05,-34.605870,-58.457702,Caballito,Departamento,3.0,2.0,60.0,52.0,149000.0,"Venta, Departamento, 3 ambientes, Balcón, coch...",74
4,RXg51b1AdPe+UXZkpRNEqg==,2021-01-07,2021-02-10,-34.587415,-58.438877,Palermo,Departamento,2.0,1.0,99.0,58.0,210000.0,U$S 210.000 - 2 amb - Palermo - Niceto Vega 52...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11947,zMNYVtyC8ZPLlHJDVEzcJg==,2021-05-22,2021-06-05,-34.562450,-58.454040,Belgrano,Departamento,3.0,2.0,76.0,70.0,204000.0,Venta Departamento 3 amb con Balcón Dependenci...,18
11948,TI8pKfusTlHy5oPeb/ZBWQ==,2021-03-02,2021-03-10,-34.598424,-58.440173,Villa Crespo,Departamento,4.0,3.0,136.0,123.0,299000.0,"4 AMB C/DEP, COCHERA, PISO ALTO! VILLA CRESPO",23
11949,y8Uvsi21S5hPqQT7OeAwAw==,2021-08-10,9999-12-31,-34.639597,-58.369096,Barracas,Departamento,2.0,1.0,105.0,92.0,230000.0,Departamento - Barracas,3
11950,R1uGoXBwjP52Z1ZAwqQ3dQ==,2021-01-11,2021-02-08,-34.574611,-58.492324,Villa Urquiza,Departamento,2.0,1.0,67.0,59.0,119538.0,Departamento - Villa Urquiza,6


In [40]:
ds_train = pd.merge(ds_train, amenities_train, on='id')
ds_train.head()

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,score_sentimientos,pileta,parrilla,balcon,sum
0,1PYAQFsAktkYlJpcdrH9uQ==,2021-03-24,2021-05-26,-34.589481,-58.389144,Recoleta,Departamento,4.0,3.0,192.0,170.0,580000.0,Balcón terraza y totalmente reciclado en Recol...,13,False,False,False,False
1,Qgy7pmJ4NulNrdbxa8a7MQ==,2021-12-13,2021-12-23,-34.555436,-58.475303,Nuñez,Departamento,3.0,2.0,83.0,83.0,100000.0,Departamento - Nuñez,11,False,True,False,False
2,QO1fBv/XWsHIqP59T0Xngg==,2021-09-24,2022-03-09,-34.577124,-58.475892,Villa Urquiza,Casa,4.0,3.0,150.0,105.0,229000.0,Casa en venta 4Amb c/patio y terraza - Urquiza R,24,False,False,True,False
3,9yo77b8vzfYAXyFJHVhLiw==,2021-07-16,9999-12-31,-34.639306,-58.368501,Barracas,Departamento,2.0,1.0,122.0,75.0,195000.0,Departamento - Barracas,34,False,False,False,True
4,HEyW45D+cF+tLUGaQPDgXg==,2021-08-30,2022-01-27,-34.563572,-58.439407,Palermo,Departamento,3.0,2.0,50.0,50.0,115000.0,"Dpto. 3 ambientes, con cocina independiente y ...",44,False,False,False,False


In [41]:
ds_test = pd.merge(ds_test, amenities_test, on='id')
ds_test.head()

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,score_sentimientos,pileta,parrilla,balcon,sum
0,poGOTUlIzEZKiMYhXKdYQQ==,2021-01-15,2021-04-29,-34.577306,-58.434056,Palermo,Departamento,3.0,2.0,68.0,68.0,158000.0,Departamento 3 Ambientes Reciclado Escritorio ...,29,False,False,False,False
1,/vmZVmXMvf5tTlS14gcC8w==,2021-09-01,2022-09-10,-34.558066,-58.473512,Belgrano,Casa,6.0,5.0,341.0,206.0,399000.0,Casa belgrano jardin pileta luminoso doble co...,16,False,False,False,False
2,ko5Ol+Mv8KN8Kglz1k/ubA==,2021-12-08,2021-12-19,-34.545531,-58.474900,Saavedra,PH,1.0,1.0,20.0,20.0,45000.0,Dpto PH Saavedra 1amb sin expe P.Baja a la cal...,38,False,False,False,True
3,/Z37wOCvzjGlLKMdOFky+Q==,2021-04-26,2021-06-05,-34.605870,-58.457702,Caballito,Departamento,3.0,2.0,60.0,52.0,149000.0,"Venta, Departamento, 3 ambientes, Balcón, coch...",74,False,False,True,False
4,RXg51b1AdPe+UXZkpRNEqg==,2021-01-07,2021-02-10,-34.587415,-58.438877,Palermo,Departamento,2.0,1.0,99.0,58.0,210000.0,U$S 210.000 - 2 amb - Palermo - Niceto Vega 52...,3,False,True,True,False


##### Expensas

Por último, trabajaremos en crear una columna numérica con el valor de las expensas. Evaluaremos primero que porcentaje de valores podemos conseguir con regex

In [42]:
expensas = re.compile(r"\s*[0-9.]*\s*exp|expensas")
calculate_freq("Expensas", expensas)

Los anuncios de propiedades que tienen la feature Expensas son: 21748 y representan el 35% de los datos


In [ ]:
expensas_extract = re.compile('((?:[a-zA-Z0-9]+\s*){5}(?:expensas|exp)\s(?:[a-zA-Z0-9]+\s){10})')
expensas_train = descriptions_train.property_description.str.extract(expensas_extract)
expensas_train.value_counts()

In [ ]:
expensas_extract = re.compile('((?:[0-9a-zA-Z,.]+\s*){5}?(?:con|sin|de)\s*(?:expensas|exp))')
expensas_train = descriptions_train.sample(100).property_description.str.extract(expensas_extract)

In [ ]:
sin_expensas = re.compile('\s*([0-9.]+)\s*exp|expensas')
descriptions_train.property_description.str.extract(sin_expensas)

## Modelos

#### Selección de features

Para entrenar los modelos usaremos nuestro dataset recien generado, descartaremos el id, el título y las fechas. Convertiremos las categóricas en variables numéricas.

In [ ]:
ds_train.columns

In [ ]:
ds_trabajo_train = ds_train.drop(['id','property_title', 'start_date', 'end_date'], axis=1)
ds_trabajo_train['place_l3'] = pd.factorize(ds_train['place_l3'])[0]
ds_trabajo_train['property_type'] = pd.factorize(ds_train['property_type'])[0]
ds_trabajo_train

In [ ]:
## Replicamos en test
ds_trabajo_test = ds_test.drop(['id','property_title', 'start_date', 'end_date'], axis=1)
ds_trabajo_test['place_l3'] = pd.factorize(ds_test['place_l3'])[0]
ds_trabajo_test['property_type'] = pd.factorize(ds_test['property_type'])[0]
ds_trabajo_test

In [ ]:
ds_trabajo_train.shape, ds_trabajo_test.shape

Sacamos la variable target y creamos nuestros datasets de entrenamiento

In [ ]:
columnas_predictoras = ds_trabajo_train.columns.to_list()
columnas_predictoras.remove('property_price')
columnas_predictoras

In [ ]:
x_train = ds_trabajo_train.loc[:, columnas_predictoras]
x_test = ds_trabajo_test.loc[:, columnas_predictoras]

y_train = ds_trabajo_train.property_price
y_test = ds_trabajo_test.property_price

In [ ]:
x_train

#### XGBoost - Regresión

best_xgb_tp1 es el arbol que tiene los mejores hiper parametros y estimaodores obtenidos en el TP1.

In [ ]:
def regression_metrics(title, real, predicted):
    mse = sk.metrics.mean_squared_error(y_true=real, y_pred=predicted)
    rmse = sk.metrics.mean_squared_error(y_true=real, y_pred=predicted, squared=False)
    r2 = sk.metrics.r2_score(y_true=real, y_pred=predicted)

    print(title)
    print(f"El error (mse) es: {mse}")
    print(f"El error (rmse) es: {rmse}")
    print(f"El error (r²) es: {r2}\n")

In [ ]:
best_xgb_tp1 = load('XGBoost-pca.joblib')

In [ ]:
best_xgb_tp1.fit(x_train_transform, y_train)
y_pred = best_xgb_tp1.predict(x_train_transform)
y_pred_test = best_xgb_tp1.predict(x_test)

##### Metricas obtenidas con el dataset del TP1.

Metricas XGBoost optimizado - Train

El error (mse) es: 761704416.3937123

El error (rmse) es: 27598.993032241455

El error (r²) es: 0.9652453545965177

---------------------------------------

Metricas XGBoost optimizado - Test

El error (mse) es: 3287902738.8784523

El error (rmse) es: 57340.23664825994

El error (r²) es: 0.8477123168506954

In [ ]:
regression_metrics("Metricas XGBoost optimizado - Train", y_train, y_pred)
regression_metrics("Metricas XGBoost optimizado - Test", y_test, y_pred_test)

Utilizamos esta pagina como soporte para entender mejor que representa cada metrica de error: 

https://sitiobigdata.com/2018/08/27/machine-learning-metricas-regresion-mse/

**Train**

Notamos como el modelo con el dataset expandido mejora significativamente.

Podemos observar un delta de **MSE** de: 432564299. Esto significa que el error se redujo un 400%. Esto es relevante ya que esta metrica nos dice sobre cuan bueno es realmente el modelo entrenado.

Luego, para **RMSE** tenemos un delta de: 9456. Esto significa que el error se redujo un 65%.

Finalmente para **R2** obtuvimos un delta de: -0,19. Esto significa que el modelo mejoró un 20%, ya que, mientras mas tengamos un valor mas cercano a uno, tenemos un modelo con un error cercano a cero.

**Test**

Notamos como el modelo con el dataset expandido mejora significativamente.

Podemos observar un delta de **MSE** de: -96136560337. Esto significa que el error en test empeoró un 3%. 

Luego, para **RMSE** tenemos un delta de: -257976,21245748526. Esto significa que el error empeoró un 550%.

Finalmente para **R2** obtuvimos un delta de: 0,52. Esto significa que el modelo empeoró un 52%.

##### Nuevos Hiperparametros optimizados con el nuevo dataset ampliado.

In [ ]:
from xgboost import XGBRegressor

#Cantidad de combinaciones que quiero porbar
n = 10

params = {
    "max_depth": [3, 12, 4],
    "learning_rate": [0.02, 0.03, 0.06],
    "min_child_weight": [2, 12, 2],
    "n_estimators": [100, 350],
    'alpha': np.linspace(0.03, 0.09, n),
}

kfold = KFold(n_splits=5)

search_regressor = XGBRegressor()

search = RandomizedSearchCV(search_regressor, params, cv=5, random_state=9, n_iter=10, verbose=10000)

search.fit(x_train_transform, y_train)

In [ ]:
# Mejores Hiperparámetros
search.best_params_

In [ ]:
# Mejor Metrica
search.best_score_

In [ ]:
best_xgb = search.best_estimator_
best_xgb.fit(x_train_transform, y_train)
y_pred = best_xgb.predict(x_train_transform)
y_pred_test = best_xgb.predict(x_test)

In [ ]:
regression_metrics("Metricas XGBoost optimizado - Train", y_train, y_pred)
regression_metrics("Metricas XGBoost optimizado - Test", y_test, y_pred_test)

**Train**

Notamos como el modelo con el dataset sigue teniendo muy buenos resultados para el dataset de Train pero con una leve baja.

Con respecto a las metricas obtenidas con el dataset del TP1, podemos observar un delta de **MSE** de: 617388041. Esto significa que el error, en comparacion con las metricas anteriores se incrementó un 287%.

Luego, para **RMSE** tenemos un delta de: 12623. Esto significa que el error se redujo un 3%.

Finalmente para **R2** obtuvimos un delta de: -0,02. Esto significa que el modelo empeoró un 2%.

**Test**

Notamos como el modelo con el dataset expandido y la busqueda de un nuevo arbol con mejores hiper-parametros, resulto en un pequeño detrimento del dataset de train pero mejoró mucho mas el dataset de test.

Con respecto a las metricas obtenidas con el dataset del TP1, podemos observar un delta de **MSE** de: 45032742345. Esto significa que el error en test mejoró un 54%. 

Luego, para **RMSE** tenemos un delta de: 82096. Esto significa que el error mejoró un 73%.

Finalmente para **R2** obtuvimos un delta de: -9795260413860356. Esto significa que el modelo mejoró un 405%.

----------------------------------------------------

Como **conclusion**, notamos que el dataset expandido nos mejora el modelo con el dataset de train un 20% pero para el dataset de test, nos lo empeora un 52%.

Cuando hicimos la busqueda de mejores hiper-parametros para el nuevo dataset expandido, logramos balancear estos resultados.

Finalmente, haciendo el delta final, el resultado de expandir el dataset nos resulto en una mejora del 18% para el dataset de train y en un 353% de mejora en el dataset de test.

### Redes Neuronales

#### Regresión

Predecir el precio de la propiedad y utilizar como métrica de evaluación el error cuadrático medio.

Vamos a predecir el precio de la propiedad (dolares) en base a la superifice total y cubierta. Ya que como vimos en el trabajo pasado, eran los atributos que mas se correlacionaban con el precio.

(Todas columnas tienen que ser numericas para Redes Neuronales)

In [ ]:
def regression_scatter(x, y_true, y_pred):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

    fig.suptitle(f"Precio según {x.name}")
    sns.scatterplot(x=x, y=y_true, ax=ax1)
    ax1.set_title(f"{x.name} vs Precio real")

    sns.scatterplot(x=x, y=y_pred, ax=ax2)
    ax1.set_title(f"{x.name} vs Precio predicho")

    sns.scatterplot(x=x, y=y_true, ax=ax3)
    sns.scatterplot(x=x, y=y_pred, ax=ax3)
    ax3.set_title(f"Grafico combinado");

In [ ]:
def plot_regression(x, y_true, y_pred, title="", xlabel="x", ylabel="y"):
    plt.figure()

    plt.plot(x, y_true, "o", label="Valores verdaderos")
    plt.plot(x, y_pred, "x", label="Valores estimados")

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
def calcular_metricas_error(title, real, predicted):
    mae = mean_absolute_error(real, predicted)
    mse = mean_squared_error(real, predicted)
    rmse = mean_squared_error(real, predicted, squared=False)
    r2 = r2_score(real, predicted)

    print(title)
    print(f"Error absoluto medio {mae}")
    print(f"Error cuadrático medio {mse}")
    print(f"Raiz del error cuadrático medio {rmse}")
    print(f"R² {r2}")

Como todas las features que tenemos están en escalas completamente diferentes y no pueden compararse, normalizaremos el dataset

In [ ]:
sscaler = StandardScaler()
x_train_transform = sscaler.fit_transform(pd.DataFrame(x_train))
x_test_transform = sscaler.fit_transform(pd.DataFrame(x_test))

#### Construcción del modelo

##### Modelo base

Entrenaremos este modelo base con una Cross validation de 10 folds

In [ ]:
d_in = 13
d_out = 1


def base_model():
    model = keras.Sequential([
        keras.layers.Dense(13, input_shape=(d_in,), kernel_initializer='normal', activation='relu'),
        keras.layers.Dense(d_out, kernel_initializer='normal')])
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
estimator = KerasRegressor(model=base_model, batch_size=5, epochs=100)
kfold = KFold(n_splits=5)
results_base = cross_val_score(estimator, x_train_transform, y_train, cv=kfold, scoring='neg_mean_squared_error')

In [ ]:
print("Baseline: %.2f (%.2f) MSE" % (results_base.mean(), results_base.std()))
print("La raíz del error cuadrático medio para el modelo base es: %.2f" % np.sqrt(abs(results_base.mean())))

Ahora probaremos lo mismo pero en un modelo un poco más grande, agregaremos una capa intermedia. Reduciremos los splits a 5 para reducir un poco el tiempo de ejecución.

##### Modelo de 3 capas

In [ ]:
def larger_model():
    model = keras.Sequential([
        keras.layers.Dense(13, input_shape=(d_in,), kernel_initializer='normal', activation='relu'),
        keras.layers.Dense(6, kernel_initializer='normal', activation='relu'),
        keras.layers.Dense(1, kernel_initializer='normal')
    ])
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
estimator = KerasRegressor(build_fn=larger_model, batch_size=5, epochs=100)
kfold = KFold(n_splits=5)
results_larger = cross_val_score(estimator, x_train_transform, y_train, cv=kfold, scoring='neg_mean_squared_error')

In [ ]:
print("El MSE %.2f (%.2f)" % (results_larger.mean(), results_larger.std()))
print(f"La raíz del error cuadrático medio para el modelo profundo es {np.sqrt(abs(results_larger.mean()))}")

Por último, probaremos un tercer modelo con una capa inical más ancha.

##### Modelo ancho

In [ ]:
def wider_model():
    model = keras.Sequential([
        keras.layers.Dense(26, input_shape=(d_in,), kernel_initializer='normal', activation='relu'),
        keras.layers.Dense(1, kernel_initializer='normal')
    ])
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
estimator = KerasRegressor(build_fn=wider_model, batch_size=5, epochs=100)
kfold = KFold(n_splits=5)
results_wider = cross_val_score(estimator, x_train_transform, y_train, cv=kfold, scoring='neg_mean_squared_error')

In [ ]:
print("El MSE %.2f (%.2f)" % (results_wider.mean(), results_wider.std()))
print(f"La raíz del error cuadrático medio para el modelo ancho es {np.sqrt(abs(results_wider.mean()))}")

#### Análisis de métricas

In [ ]:
print("La raíz del error cuadrático medio para el modelo base es: %.2f" % np.sqrt(abs(results_base.mean())))
print("La raíz del error cuadrático medio para el modelo profundo es: %.2f" % np.sqrt(abs(results_larger.mean())))
print(f"La raíz del error cuadrático medio para el modelo ancho es {np.sqrt(abs(results_wider.mean()))}")

Analizaremos un poco más en detalle el modelo profundo que tiene mejor métrica.

In [ ]:
model = larger_model()

model.fit(x_train_transform, y_train)

In [ ]:
y_pred = model.predict(x_train_transform)
y_pred_test = model.predict(x_test_transform)

In [ ]:
precios_train = pd.concat([y_train, pd.DataFrame(y_pred, columns=['predicted'])], axis=1)
precios_test = pd.concat([y_test, pd.DataFrame(y_pred_test, columns=['predicted'])], axis=1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))
fig.suptitle("Distribución de precios reales y predichos Train")

sns.kdeplot(precios_train['property_price'], ax=ax1)
sns.kdeplot(y_pred[:,0], ax=ax2)

Podemos ver que la campana real es bastante similar a la predicha tanto en train como en test. La campana predicha es levemente más baja habiendo menos densidad en los precios medios.

In [ ]:
calcular_metricas_error("Metricas de train:", y_train, y_pred)
calcular_metricas_error("Metricas de test:", y_test, y_pred_test)

In [ ]:
regression_scatter(ds_train.property_surface_covered, y_train, y_pred[:, 0])
regression_scatter(ds_train.property_surface_total, y_train, y_pred[:, 0])
regression_scatter(ds_train.latitud, y_train, y_pred[:, 0])
regression_scatter(ds_train.longitud, y_train, y_pred[:, 0])

#### Clasificación

##### Construcción del target

In [ ]:
scaler = StandardScaler()
# Droppeamos todas las columnas del dataset que no vamos a utilizar
x_train = train_ds.drop(axis = 1, columns = ["start_date", 
                                             "end_date", 
                                             "place_l3",
                                             "property_price" ,
                                             "property_type", 
                                             "property_title", 
                                             "pxm2", 
                                             'tipo_precio'])
# Escalamos los datos
x_train = scaler.fit_transform(x_train)

# Construimos el target con la variable objetivo
y_train = train_ds.tipo_precio
y_train = np.array(y_train)

# Repetimos los pasos con el dataset de test
x_test = test_ds.drop(axis=1, columns= ['place_l3', 
                       'property_title', 
                       'property_type', 
                       'property_price', 
                       'tipo_precio',
                       'pxm2'])

x_test = scaler.fit_transform(x_test)
y_test = test_ds.tipo_precio
y_test = np.array(y_test)


# Realizamos el one hot encoder para transformar la variable target en numérica tanto en train como test
enc = OneHotEncoder()
y_train_encoded = enc.fit_transform(y_train[:, np.newaxis]).toarray()
y_test_encoded = enc.transform(y_test[:, np.newaxis]).toarray()

##### Modelo

Optamos por agregar una capa intermedia de 6 neuronas con la función de activación reLU. Luego una capa de 4 neuronas con función tanh. Finalmente una capa de salida de 3 neuronas con la función de activación sigmoidea ya que se trata de un problema de clasificación.

In [ ]:
cantidad_de_posibles_respuestas=len(np.unique(y_train))
cantidad_de_variables_predictoras=len(x_train[0])


modelo = keras.Sequential([
    keras.layers.Dense(6,input_shape=(cantidad_de_variables_predictoras,), activation='relu'),
    keras.layers.Dense(4,activation= 'tanh'),
    keras.layers.Dense(cantidad_de_posibles_respuestas,input_shape=(cantidad_de_variables_predictoras,),activation='sigmoid')])

modelo.summary()

In [ ]:
Como optimizador decidimos utilizar Adam con un learning rate lo suficientemente bajo como para no realizar saltos demasiado grandes a la hora de converger.

In [ ]:
modelo.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    # Elegimos la siguiente función ya que se trata de una red neuronal de clasificación
  loss='categorical_crossentropy',
    # Como métrica elegimos accuracy
  metrics=['accuracy'], 
)

# La cantidad de Epochs está en 300 para ver si mejoraban las métricas, se pueden cambiar a 100 tranquilamente 
# para que ejecute más rápido
cant_epochs= 100

historia_entrenamiento_modelo = modelo.fit(x_train,y_train_encoded,epochs=cant_epochs,batch_size=16,verbose=True, workers= -1, use_multiprocessing=True)

In [ ]:
output_modelo = modelo.predict(x_train)

predicciones = np.argmax(output_modelo, axis=1).tolist()
valores_esperados = np.argmax(y_train_encoded, axis=1).tolist()


tabla_iris = confusion_matrix(predicciones, valores_esperados)
sns.heatmap(tabla_iris,annot=True, cmap = 'Blues', fmt= 'g').set(title='Predicciones sobre el conjunto de entrenamiento')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
print(classification_report(predicciones, valores_esperados))

In [ ]:
output_modelo = modelo.predict(x_test)

predicciones = np.argmax(output_modelo, axis=1).tolist()
valores_esperados = np.argmax(y_test_encoded, axis=1).tolist()


tabla_iris = confusion_matrix(predicciones, valores_esperados)
sns.heatmap(tabla_iris,annot=True, cmap = 'Blues', fmt= 'g').set(title='Predicciones sobre el conjunto de testeo')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
print(classification_report(predicciones, valores_esperados))

Vemos como en los primeros 10 epochs rápidamente se progresa fuertemente la métric de accuracy y pasado eso la mejora es paulatina

In [ ]:
epochs = range(cant_epochs)

plt.plot(epochs, historia_entrenamiento_modelo.history['accuracy'], color='orange', label='accuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()

En el caso del minimizado del error vemos como en los primeros 10 epochs se reduce fuertemente. Entre el epoch 10 y 40 a una menor medida y pasado el epoch 40 la reducción es mínima.

In [ ]:
epochs = range(cant_epochs)

plt.plot(epochs, historia_entrenamiento_modelo.history['loss'], color='orange', label='loss')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()

In [ ]:
### DS Expandido:

### top3 > % regex -> booleanas
### expensas sacadas con regex --> numerico
### property_desc_score = tecnica chino -> numerico
### serian 5 columnas en total

Otro factor interesante que podriamos analizar es a que tipo de vivienda pertenece cada ammenity. Y si donde encontramos una ammenity en particular, encontramos consecuentemente otra. Por ejemplo, una casa con jardin y parrila y/o pileta.
De esta manera podriamos tratar de determinar el costo de cada ammenity o como afecta al precio.

## Ensambles de modelos

### Ensamble 1

### Ensamble 2

## Conclusiones